In [1570]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from shiny import ui, render, App
import warnings
warnings.filterwarnings('ignore')

# greenness data raw: https://open.canada.ca/data/en/dataset/d61c2113-e786-4767-8262-22069efe428e

# import the data into a dataframe
df = pd.read_csv(r'data/NDVI_data.csv')
df.head()

,REF_DATE,GEO,DGUID,Urban greenness,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2000,"Canada, all population centres",NaN,Average greenness,Percent,239,units,0,v1446860218,1.1,83.7000,NaN,NaN,NaN,1
1,2000,"Canada, all population centres",NaN,Average normalized difference vegetation index...,Index,160,units,0,v1446860219,1.2,0.6449,NaN,NaN,NaN,4
2,2000,"Canada, large urban population centres",NaN,Average greenness,Percent,239,units,0,v1446860220,2.1,80.5000,NaN,NaN,NaN,1
3,2000,"Canada, large urban population centres",NaN,Average normalized difference vegetation index...,Index,160,units,0,v1446860221,2.2,0.6282,NaN,NaN,NaN,4
4,2000,"Canada, medium population centres",NaN,Average greenness,Percent,239,units,0,v1446860222,3.1,83.6000,NaN,NaN,NaN,1


In [1571]:
# remove the unwanted columns
df = df[['REF_DATE', 'GEO', 'Urban greenness','VALUE']]

# Create unique indexers from combination of the "GEO" (population center) column and the "REF_DATE" (year) columns, and set as the index
df['index'] = df['GEO'] + ', ' + df['REF_DATE'].astype('str')
df.set_index('index', inplace=True)

In [1572]:
# The "Urban Greenness" column contains alternating values for "Average greenness" and "Average normalized difference vegetation index" 
# (NDVI), with respective values in the "VALUE" column
# The data is sorted such that every two rows for these variables and their respective values pertain to a single population 
# area (e.g. Canada, Ontario, Hamilton...) (i.e. there is one observation for every two rows)
# As such, "Average greenness" and "Average normalized difference vegetation index" and their respective values in "VALUES" will be 
# re-assigned under a new column each, effectively reducing the rows by half while preserving the number of observations (population areas):

# Create new dataframes containing Average Greenness and Average normalized difference vegetation index (NDVI)... rows
df_avg_grn = df[df['Urban greenness'].str.contains('Average greenness', case=False)]
df_ndvi = df[df['Urban greenness'].str.contains('normalized difference vegetation index', case=False)]

# rename "VALUE" columns to preserve upon rejoining
df_avg_grn = df_avg_grn.rename(columns={'VALUE':'Avg Greenness'})
df_ndvi = df_ndvi.rename(columns={'VALUE':'NDVI'})

# remove unwanted columns before rejoining
df_avg_grn = df_avg_grn[['REF_DATE', 'GEO', 'Avg Greenness']]
df_ndvi = df_ndvi[['NDVI']]

# Rejoin (concatenate) datframes based on the index created in the previous step
df = pd.concat([df_avg_grn, df_ndvi], axis=1)

# Remove unused dataframes to save memory
del df_avg_grn
del df_ndvi

In [1573]:
# The data itself already contains aggregate measures for the province and country - these will be removed

# Find a list of indexers pertaining to the aggregate measures
# These can be indentified within the index as keys contining the text 'centres', as opposed to 'centre' which is observed in the 
# non-aggregate observation keys (i.e. induvidual cities/population centres)
agg_list = df[df['GEO'].str.contains('centres')].index

# remove from original dataframe
df = df.loc[~df.index.isin(agg_list)]

In [1574]:
# Now the index (and "GEO") contains only induvidual population centres that share the format: 
# "[City], [Province/Territory], [small/med/large] [urban] population centre, [Year]"

# As such, new columns will be created for the city, province/territory, population centre size, and urban [yes/no] as categorical variables
# Since these variable are delimited by commas in "GEO", grab the text before, between, and after the commas and assign each to a new column:

df['Municipality'] = df.loc[:,'GEO'].str.split(',').str[0] # The Population Centre name is text before the first comma
# Some municipalities are combined with smaller metropolitan/adjacent municipalities
# The larger/predominant municipality is always listed first, before the ' - ', this will be extracted
df['Municipality'] = df.loc[:,'Municipality'].str.split('\s-\s').str[0] 
df['Province'] = df.loc[:,'GEO'].str.split(', ').str[1].str.split(', ').str[0] # Province is located between the two commas

def map_size(string):
    # Mapping function that returns S, M, or L (or none) is 'small', 'medium', or 'large' is found in a string
    if 'small' in string:
        return 'Small'
    elif 'medium' in string:
        return 'Medium'
    elif 'large' in string:
        return 'Large'
    else:
        return None 

# Apply the above function to each observation at the "GEO" string column, which will contain one of 'small', 'medium', or 'large'
df['Municipality Size'] = df['GEO'].apply(map_size)

def map_class(string):
    # Mapping function that returns land class (urban or rural) based on string contents
    if 'urban' in string:
        return 'Urban'
    else:
        return 'Rural'

# Create a boolean layer (1 = urban, 0 = non-urban) based on if the text "urban" is present in "GEO"
df['Land Class'] =  df['GEO'].apply(map_class)

# Now that we have pulled the information out of "GEO", it can be dropped
df = df.drop(columns='GEO')

# We will also remove rows that have an 'NaN' value(s)
df = df.dropna()

In [1575]:
# Lastly, now that the data has been cleaned and transformed, we will create a multi-level index
# This will help in dataframe slicing to produce regional or temporal aggregates
# The multi-level heirarchical index is based on the Province, Pop. Centre, and Year
df = df.set_index(['Province', 'Municipality', 'REF_DATE'])

In [1576]:
# Can now slice by Province...
#df.loc['British Columbia']

# or by City...
#df.loc['British Columbia','Vancouver']

# or by Year
#df.loc[:,:,2000]

# or multiple...e.g. find the 'Large' sized cities in Ontario during the year 2004
df.loc['Ontario',:,2004][df.loc['Ontario',:,2004]['Municipality Size'] == 'Large']

,Avg Greenness,NDVI,Municipality Size,Land Class
Municipality,,,,
Barrie,86.4,0.6535,Large,Urban
Brantford,86.1,0.6264,Large,Urban
Guelph,88.0,0.6562,Large,Urban
Hamilton,82.5,0.6256,Large,Urban
Kanata,97.4,0.6931,Large,Urban
Kingston,95.1,0.6802,Large,Urban
Kitchener,86.5,0.6498,Large,Urban
London,89.5,0.6526,Large,Urban
Milton,76.3,0.5836,Large,Urban


In [1577]:
# The categorical 'Population Centre Size' variable is not very informative, given it only has 3 classes (small, med, lrg)
# To further explore the effects of municipality size (population) on greenness, this will be supplemented by more-detailed population data
# This new data (also from Gov. Canada) is of very similar format to the previous NDVI/Avg Greenness data:
#     - data is provided for population centres with a population >5000
#     - data is provided on an annualized basis
#     - some aggregate-data rows exist
#     - a "GEO" column is present with the province and name of the population centre
#     - a "DGUID" column exists, which is a census-specific identification key...unfortunately these do not align b/w datasets
# As such, the data will be joined based on the name and province of the population centres
# Some data lossess are expected
#
# population data raw: 
# https://open.canada.ca/data/en/dataset/6841ba54-09d3-4c12-a2fc-a5064694a860/resource/92c62a9b-4ac0-4168-89af-39a2c477b4b0

In [1578]:
# Load the population dataset
df_pop = pd.read_csv(r'data/pop_data_17100142.csv')

# Initial data cleaning and re-indexing, similar to previous itteration
# Remove all rows with no census id (DGUID) values - these are some of the pre-existing aggregate rows
df_pop = df_pop.loc[~df_pop['DGUID'].isna()]

# Remove all data for townships ('TP') - these are not represented in the NDVI/greenness dataset
# This is necessary because there are some townships with the same name as cities (which are all included in the NDVI/greenness dataset)
# For instance, there is a city named 'Hamilton' and a much smaller and unrelated township named 'Hamilton', both in Ontario
# Removing these townships resolves duplication issues prior to joining with teh NDVI/greenness dataframe
df_pop = df_pop.loc[~df_pop['GEO'].str.contains('(TP)')]

# Removing unwanted columns
df_pop = df_pop[['REF_DATE', 'GEO', 'VALUE', 'DGUID']]

# Rename 'Value' column
df_pop.rename(columns={'VALUE':'Population'}, inplace=True)

In [1579]:
# Extract the population centre name from the "GEO" columns string
#      - it appears before the first ' (' in each row
df_pop['Municipality'] = df_pop.loc[:,'GEO'].str.split('\s\(').str[0]
df_pop['Municipality'] = df_pop.loc[:,'Municipality'].str.split(',').str[0]

# Remove the remaining aggregate rows (province and country totals)
drop_list = df.index.levels[0].to_list() + ['Canada'] # get list of all provinces plus 'Canada'
df_pop = df_pop.loc[~df_pop['Municipality'].isin(drop_list)]

In [1580]:
# Extract the Province name - appears after the last column in each row
df_pop['Province'] = df_pop.loc[:,'GEO'].str.split(', ').str[-1]

# Remove now-redundant "GEO" and "DGUID" columns
df_pop = df_pop.drop(['GEO','DGUID'], axis=1)

# set an identical multi-index for which to join dataframes on
df_pop = df_pop.set_index(['Province', 'Municipality', 'REF_DATE'])

In [1581]:
# Merge the two dataframes, preserve all rows (for now) using an 'outer' merge
df2 = pd.merge(df, df_pop, left_index=True, right_index=True, how='outer')

# remove all of the rows that do not contain full set of observations
df2 = df2.dropna()

In [1582]:
# Rapidly expanding population centres can be reasonably expected to see a greater reduction in greenspace
# This warrants further exploration...
# We will create a categorical variable describing expanding, stable, and contracting population centres
# This will be based on the average yoy % growth since 2001

In [1583]:
# Group each city in the index and calculate the year over year % change in its population
df2['YoY Population Change %'] = df2.groupby(level=1)['Population'].pct_change() * 100

# ...and do the same for NDVI and greenness values
df2['YoY NDVI Change %'] = df2.groupby(level=1)['NDVI'].pct_change() * 100
df2['YoY Greenness Change %'] = df2.groupby(level=1)['Avg Greenness'].pct_change() * 100
df2.head()

Avg Greenness    NDVI Municipality Size  \
Province Municipality REF_DATE                                            
Alberta  Airdrie      2001               77.7  0.6068            Medium   
                      2002               59.8  0.5254            Medium   
                      2003               66.7  0.5764            Medium   
                      2004               68.8  0.5846            Medium   
                      2005               61.4  0.5633            Medium   

                               Land Class  Population  \
Province Municipality REF_DATE                          
Alberta  Airdrie      2001          Rural     21641.0   
                      2002          Rural     23374.0   
                      2003          Rural     24946.0   
                      2004          Rural     26613.0   
                      2005          Rural     28610.0   

                                YoY Population Change %  YoY NDVI Change %  \
Province Municipality REF_DATE                                               
Alberta  Airdrie      2001                          NaN                NaN   
                      2002                     8.007948         -13.414634   
                      2003                     6.725421           9.706890   
                      2004                     6.682434           1.422623   
                      2005                     7.503852          -3.643517   

                                YoY Greenness Change %  
Province Municipality REF_DATE                          
Alberta  Airdrie      2001                         NaN  
                      2002                  -23.037323  
                      2003                   11.538462  
                      2004                    3.148426  
                      2005                  -10.755814

In [1584]:
# Quantiles of the population growth data will determine growth rate categories
# Anything negative will be assigned 'contracting population', as such, we are only considering everything above zero
# Each Category will have one-third of the positive growth rate municipalities
df2['YoY Population Change %'].loc[df2['YoY Population Change %']>=0].quantile([.33,.67,])

0.33    0.903364
0.67    2.344959
Name: YoY Population Change %, dtype: float64

In [1585]:
# Calculate the mean growth rate (since 2001) of each city by grouping the data by cities (index level 1) by passing
# it through a mapping function, which returns the respective category based on the resulting mean growth rate

def map_growth_class(city):
    # Mapping function that returns growth class based on mean yoy growth for city
    mean = city.mean() # calc mean population growth
    # assign the categories based on the mean value
    if mean < 0:
        return 'Contracting Population (<0.0%)' # contracting populations for negative avg pop. change
    elif 0 <= mean < 0.90:
        return 'Low Population Growth (<0.9%)' # for growth under 0.9%
    elif 0.90 <= mean < 2.34:
        return 'Moderate Population Growth (<2.34%)' # for growth between 2.34%
    elif mean >= 2.34:
        return 'Rapid Population Growth (>2.34%)' # for growth over 2.34%
    else:
        return None # in case of missing data, errors, nan etc...

# apply the function to the population change column for each city (index level 1)
df2['Growth Class'] = df2.groupby(level=1)['YoY Population Change %'].transform(map_growth_class)


In [1586]:
# Create new column to specify the current population and NDVI values
# This will be used in a later step to sort data analgous to the 'fct_reorder2'function in R
df2['Current Population'] = df2.groupby(level=1)['Population'].transform('last')
df2['Current NDVI'] = df2.groupby(level=1)['NDVI'].transform('last')
df2['Current Avg Greenness'] = df2.groupby(level=1)['Avg Greenness'].transform('last')

In [1587]:
# Save the final dataset - Now ready for the Shiny App
df2.to_csv('cleaned_data.csv')

In [1588]:
# Shiny App code developed in seperate IDE
# 'cleaned_data.csv' is imported and used for plotting in the Shiny app.py file